# PLOT HOURLY DATA ACROSS ENTIRE US

## for new parameter change: 

- param
- cmap color
- set_clim
- colorbar ticks
- file name when making into gif

In [1]:
from metpy.plots import USSTATES, USCOUNTIES
import cartopy.crs as ccrs
from shapely.geometry import Point 
from matplotlib.cm import ScalarMappable
import pandas as pd
import matplotlib.pyplot as plt
import os

year_dir = "/media/kaleb/extraSpace/crop/daily_data/2022/"
yyyymmdd = "20220101"
param = 'Avg Temperature (K)'

all_data = []
month = yyyymmdd[4:6]
day = yyyymmdd[6:8]

for state_dir in os.listdir(year_dir):
    state_path = os.path.join(year_dir, state_dir)

    # iteratore over the files in the dir
    for file in os.listdir(state_path):

        # if the filename has the month we're looking for
        if month in file:
            file_path = os.path.join(state_path, file) # full path to file
            df = pd.read_csv(file_path, index_col=False)

            # to minimize space, only append the necessary columns
            df['Lat'] = (df['Lat (llcrnr)'] + df['Lat (urcrnr)']) / 2
            df['Lon'] = (df['Lon (llcrnr)'] + df['Lon (urcrnr)']) / 2

            # df['Day'] = pd.to_numeric(df['Day']) # convert to int if stored as str
            # df = df[df['Day'] == day]

            df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])

            df = df[['Date', 'Lat', 'Lon', param]]
            df = df.set_index('Date')
            df = df['2022-01-01':'2022-01-04']
            df.reset_index(inplace=True)
            all_data.append(df)

combined_df = pd.concat(all_data)
combined_df = combined_df.reset_index(drop=True)

df = combined_df
df = df.sort_values(by=['Date'])


# chop it down to first 5 days
new_df = df.set_index('Date')
new_df = new_df['2022-01-01':'2022-01-04']
new_df.reset_index(inplace=True)
df = new_df
df = df.sort_values(by='Date')

# made an array of hour dfs
df_arr = [group for _, group in df.groupby(pd.Grouper(key='Date', freq='H'))]

lon_min, lon_max = combined_df['Lon'].min(), combined_df['Lon'].max()
lat_min, lat_max = combined_df['Lat'].min() - 3, combined_df['Lat'].max()

# calculate the average to find the center
lat_avg, lon_avg = combined_df['Lat'].mean(), combined_df['Lon'].mean()
extent = [lon_min, lon_max, lat_min, lat_max]


i=0
for d in df_arr:
    proj = ccrs.LambertConformal(central_longitude=lon_avg, central_latitude=lat_avg) # center the spherical orientation
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1, projection=proj)
    ax.set_extent(extent, ccrs.Geodetic())
    ax.add_feature(USSTATES.with_scale('5m'))

    cmap = plt.cm.get_cmap('jet') # viridis, plasma, cool, jet, twilight_shifted, greys, blues
    ax.scatter(d['Lon'], d['Lat'], c=d[param],
            cmap=cmap, transform=ccrs.PlateCarree(), s=1)

    sm = ScalarMappable(cmap=cmap)
    sm.set_array(d[param])
    sm.set_clim(vmin=250, vmax=310)
    plt.colorbar(sm, ax=ax, pad=0.01, shrink=0.55, ticks=[250, 260, 270, 280, 290, 300, 310], 
                label='Temperature (K)')
    plt.title('Temperature (K)\nDate: ' + str(d['Date'].iloc[0]))
    plt.tight_layout()
    plt.savefig(f'wrf/{i:003}', )
    plt.close()
    i+=1

In [2]:
!convert -delay 20 wrf/*.png wrf_tmp.gif

In [9]:
year_dir = "/media/kaleb/extraSpace/crop/daily_data/2022/"
yyyymmdd = "20220101"
param = 'Precipitation (kg m**-2)'

all_data = []
month = yyyymmdd[4:6]
day = yyyymmdd[6:8]

for state_dir in os.listdir(year_dir):
    state_path = os.path.join(year_dir, state_dir)

    # iteratore over the files in the dir
    for file in os.listdir(state_path):

        # if the filename has the month we're looking for
        if month in file:
            file_path = os.path.join(state_path, file) # full path to file
            df = pd.read_csv(file_path, index_col=False)

            # to minimize space, only append the necessary columns
            df['Lat'] = (df['Lat (llcrnr)'] + df['Lat (urcrnr)']) / 2
            df['Lon'] = (df['Lon (llcrnr)'] + df['Lon (urcrnr)']) / 2

            # df['Day'] = pd.to_numeric(df['Day']) # convert to int if stored as str
            # df = df[df['Day'] == day]

            df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])

            df = df[['Date', 'Lat', 'Lon', param]]
            all_data.append(df)

combined_df = pd.concat(all_data)
combined_df = combined_df.reset_index(drop=True)

df = combined_df
df = df.sort_values(by=['Date'])


# chop it down to first 5 days
new_df = df.set_index('Date')
new_df = new_df['2022-01-01':'2022-01-04']
new_df.reset_index(inplace=True)
df = new_df

# made an array of hour dfs
df_arr = [group for _, group in df.groupby(pd.Grouper(key='Date', freq='H'))]

lon_min, lon_max = combined_df['Lon'].min(), combined_df['Lon'].max()
lat_min, lat_max = combined_df['Lat'].min() - 3, combined_df['Lat'].max()

# calculate the average to find the center
lat_avg, lon_avg = combined_df['Lat'].mean(), combined_df['Lon'].mean()
extent = [lon_min, lon_max, lat_min, lat_max]


i=0
for d in df_arr:
    proj = ccrs.LambertConformal(central_longitude=lon_avg, central_latitude=lat_avg) # center the spherical orientation
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1, projection=proj)
    ax.set_extent(extent, ccrs.Geodetic())
    ax.add_feature(USSTATES.with_scale('5m'))

    cmap = plt.cm.get_cmap('Blues') # viridis, plasma, cool, jet, twilight_shifted, greys, blues
    ax.scatter(d['Lon'], d['Lat'], c=d[param],
            cmap=cmap, transform=ccrs.PlateCarree(), s=1)

    sm = ScalarMappable(cmap=cmap)
    sm.set_array(d[param])
    sm.set_clim(vmin=0, vmax=10)
    plt.colorbar(sm, ax=ax, pad=0.01, shrink=0.55, ticks=[0, 5, 10], 
                label=param)
    plt.title(param + '\nDate: ' + str(d['Date'].iloc[0]))
    plt.tight_layout()
    plt.savefig(f'wrf/{i:003}')
    plt.close()
    i+=1

In [10]:
!convert -delay 20 wrf/*.png wrf_precip.gif

In [5]:
year_dir = "/media/kaleb/extraSpace/crop/daily_data/2022/"
yyyymmdd = "20220101"
param = 'Downward Shortwave Radiation Flux (W m**-2)'

all_data = []
month = yyyymmdd[4:6]
day = yyyymmdd[6:8]

for state_dir in os.listdir(year_dir):
    state_path = os.path.join(year_dir, state_dir)

    # iteratore over the files in the dir
    for file in os.listdir(state_path):

        # if the filename has the month we're looking for
        if month in file:
            file_path = os.path.join(state_path, file) # full path to file
            df = pd.read_csv(file_path, index_col=False)

            # to minimize space, only append the necessary columns
            df['Lat'] = (df['Lat (llcrnr)'] + df['Lat (urcrnr)']) / 2
            df['Lon'] = (df['Lon (llcrnr)'] + df['Lon (urcrnr)']) / 2

            # df['Day'] = pd.to_numeric(df['Day']) # convert to int if stored as str
            # df = df[df['Day'] == day]

            df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])

            df = df[['Date', 'Lat', 'Lon', param]]
            all_data.append(df)

combined_df = pd.concat(all_data)
combined_df = combined_df.reset_index(drop=True)

df = combined_df
df = df.sort_values(by=['Date'])


# chop it down to first 5 days
new_df = df.set_index('Date')
new_df = new_df['2022-01-01':'2022-01-04']
new_df.reset_index(inplace=True)
df = new_df

# made an array of hour dfs
df_arr = [group for _, group in df.groupby(pd.Grouper(key='Date', freq='H'))]

lon_min, lon_max = combined_df['Lon'].min(), combined_df['Lon'].max()
lat_min, lat_max = combined_df['Lat'].min() - 3, combined_df['Lat'].max()

# calculate the average to find the center
lat_avg, lon_avg = combined_df['Lat'].mean(), combined_df['Lon'].mean()
extent = [lon_min, lon_max, lat_min, lat_max]


i=0
for d in df_arr:
    proj = ccrs.LambertConformal(central_longitude=lon_avg, central_latitude=lat_avg) # center the spherical orientation
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1, projection=proj)
    ax.set_extent(extent, ccrs.Geodetic())
    ax.add_feature(USSTATES.with_scale('5m'))

    cmap = plt.cm.get_cmap('Greys') # viridis, plasma, cool, jet, twilight_shifted, greys, blues
    ax.scatter(d['Lon'], d['Lat'], c=d[param],
            cmap=cmap, transform=ccrs.PlateCarree(), s=1)

    sm = ScalarMappable(cmap=cmap)
    sm.set_array(d[param])
    sm.set_clim(vmin=0, vmax=700)
    plt.colorbar(sm, ax=ax, pad=0.01, shrink=0.55, ticks=[0, 175, 350, 525, 700], 
                label=param)
    plt.title(param + '\nDate: ' + str(d['Date'].iloc[0]))
    plt.tight_layout()
    plt.savefig(f'wrf/{i:003}')
    plt.close()
    i+=1

In [6]:
!convert -delay 20 wrf/*.png wrf_rad.gif

In [13]:
year_dir = "/media/kaleb/extraSpace/crop/daily_data/2022/"
yyyymmdd = "20220101"
param = 'Relative Humidity (%)'

all_data = []
month = yyyymmdd[4:6]
day = yyyymmdd[6:8]

for state_dir in os.listdir(year_dir):
    state_path = os.path.join(year_dir, state_dir)

    # iteratore over the files in the dir
    for file in os.listdir(state_path):

        # if the filename has the month we're looking for
        if month in file:
            file_path = os.path.join(state_path, file) # full path to file
            df = pd.read_csv(file_path, index_col=False)

            # to minimize space, only append the necessary columns
            df['Lat'] = (df['Lat (llcrnr)'] + df['Lat (urcrnr)']) / 2
            df['Lon'] = (df['Lon (llcrnr)'] + df['Lon (urcrnr)']) / 2

            # df['Day'] = pd.to_numeric(df['Day']) # convert to int if stored as str
            # df = df[df['Day'] == day]

            df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])

            df = df[['Date', 'Lat', 'Lon', param]]
            all_data.append(df)

combined_df = pd.concat(all_data)
combined_df = combined_df.reset_index(drop=True)

df = combined_df
df = df.sort_values(by=['Date'])


# chop it down to first 5 days
new_df = df.set_index('Date')
new_df = new_df['2022-01-01':'2022-01-04']
new_df.reset_index(inplace=True)
df = new_df
df 


,Date,Lat,Lon,Relative Humidity (%)
0,2022-01-01 00:00:00,34.373874,-88.127634,79.7
1,2022-01-01 00:00:00,38.804139,-78.902161,59.7
2,2022-01-01 00:00:00,38.898595,-94.775774,65.0
3,2022-01-01 00:00:00,39.005590,-94.775774,70.8
4,2022-01-01 00:00:00,38.252584,-78.809681,52.8
...,...,...,...,...
9701371,2022-01-04 23:00:00,48.318796,-99.275049,81.5
9701372,2022-01-04 23:00:00,35.509603,-95.203433,32.8
9701373,2022-01-04 23:00:00,38.642301,-105.966152,32.9
9701374,2022-01-04 23:00:00,35.403999,-99.131979,29.4


In [ ]:

# made an array of hour dfs
df_arr = [group for _, group in df.groupby(pd.Grouper(key='Date', freq='H'))]

lon_min, lon_max = combined_df['Lon'].min(), combined_df['Lon'].max()
lat_min, lat_max = combined_df['Lat'].min() - 3, combined_df['Lat'].max()

# calculate the average to find the center
lat_avg, lon_avg = combined_df['Lat'].mean(), combined_df['Lon'].mean()
extent = [lon_min, lon_max, lat_min, lat_max]


i=0
for d in df_arr:
    proj = ccrs.LambertConformal(central_longitude=lon_avg, central_latitude=lat_avg) # center the spherical orientation
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1, projection=proj)
    ax.set_extent(extent, ccrs.Geodetic())
    ax.add_feature(USSTATES.with_scale('5m'))

    cmap = plt.cm.get_cmap('Oranges') # viridis, plasma, cool, jet, twilight_shifted, greys, blues
    ax.scatter(d['Lon'], d['Lat'], c=d[param],
            cmap=cmap, transform=ccrs.PlateCarree(), s=1)

    sm = ScalarMappable(cmap=cmap)
    sm.set_array(d[param])
    sm.set_clim(vmin=0, vmax=100)
    plt.colorbar(sm, ax=ax, pad=0.01, shrink=0.55, ticks=[0, 25, 50, 75, 100], 
                label=param)
    plt.title(param + '\nDate: ' + str(d['Date'].iloc[0]))
    plt.tight_layout()
    plt.savefig(f'wrf/{i:003}')
    plt.close()
    i+=1

In [12]:
!convert -delay 20 wrf/*.png wrf_humid.gif